In [ ]:
# ! pip install sounddevice 

In [ ]:
import torch
import sounddevice as sd
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import numpy as np
import scipy.signal

# Carrega o modelo e o processador
model_id = "freds0/distil-whisper-large-v3-ptbr"
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model.eval()

In [4]:
# ! pip install faster-whisper


In [ ]:
from faster_whisper import WhisperModel

model = WhisperModel("medium")

c:\Users\nielsen.castelo\AppData\Local\anaconda3\envs\GPU_Toch\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nielsen.castelo\.cache\huggingface\hub\models--Systran--faster-whisper-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from faster_whisper import WhisperModel
import sounddevice as sd
import numpy as np
import tempfile
import scipy.io.wavfile

# Parâmetros de gravação
duration = 5  # segundos
sample_rate = 16000

print("🎙️ Gravando áudio...")
audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
sd.wait()
print("✅ Gravação concluída.")

# Salva o áudio temporariamente como WAV
audio = np.squeeze(audio)
audio_int16 = (audio * 32767).astype(np.int16)

with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
    scipy.io.wavfile.write(tmp.name, sample_rate, audio_int16)
    audio_path = tmp.name

# Carrega o modelo (você pode usar "base", "small", "medium", "large-v2", etc)
model = WhisperModel("medium", compute_type="float32", device="cpu")  # ou device="cuda"

# Transcrição
segments, info = model.transcribe(audio_path, language="pt")

print(f"\n📝 Transcrição (detected language: {info.language})")
for segment in segments:
    print(f"[{segment.start:.2f}s - {segment.end:.2f}s] {segment.text}")
